In [212]:
import matplotlib.pyplot as plt
from matplotlib import cm
from time import time
import numpy as np
import math
import numpy.linalg as la
%matplotlib qt

In [213]:
def F_U(x):
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]-1)**2)-1)**2 -(20*x[0]+40*x[1])
    return fx

In [214]:
#Primera derivada utilizando la derivada central
def gradiente(x,delta):
    grad=np.zeros(2)
    grad[0]=(F_U([x[0]+delta,x[1]])- F_U([x[0]-delta,x[1]]))/(2*delta)
    grad[1]=(F_U([x[0],x[1]+delta])- F_U([x[0],x[1]-delta]))/(2*delta)
    return grad

In [215]:
#Segunda derivada utilizando la derivada central
def Hs(x,delta):
    H=np.zeros([2,2])
    H[0,0]= (F_U([x[0]+delta,x[1]])  - 2*F_U(x) + F_U([x[0]-delta,x[1]]) )/ delta**2;
    H[1,1]= (F_U([x[0],x[1]+delta])  - 2*F_U(x) + F_U([x[0],x[1]-delta]) )/ delta**2; 
    H[0,1]= (F_U([x[0]+delta,x[1]+delta]) - F_U([x[0]+delta,x[1]-delta]) - F_U([x[0]-delta,x[1]+delta]) + F_U([x[0]-delta,x[1]-delta]) )/ (4*(delta**2));
    H[1,0]=H[0,1]
    return H


In [216]:
#Evaluar la función con el punto mínimo y [-3,2]
#
x=[-3,2]
print(x[0])
print(x[1])
len(x)
print("F_U evaluada en :" ,x,"{:.4f}".format(F_U(x)))


-3
2
F_U evaluada en : [-3, 2] 1452.2619


In [217]:
#Verificar el valor mínimo de la función
x=[0.5000,0.1200]
print("F_U evaluada en :" ,x,"{:.4f}".format(F_U(x)))

F_U evaluada en : [0.5, 0.12] -9.6547


In [218]:
#Definir la función para la sección dorada
def GoldenSection(x, search):
    a = -5
    b = 5
    tau = 0.381967
    epsilon = 1e-5
    alpha1 = a*(1 - tau) + b*tau
    alpha2 = a*tau + b*(1 - tau)
    falpha1 = F_U(x + alpha1*search)
    falpha2 = F_U(x + alpha2*search)
    for _ in range(0, 1000):
        if falpha1 > falpha2:
            a = alpha1
            alpha1 = alpha2
            falpha1 = falpha2
            alpha2 = tau*a + (1 - tau)*b
            falpha2 = F_U(x + alpha2*search)
        else:
            b = alpha2
            alpha2  = alpha1
            falpha2 = falpha1
            alpha1  = tau*b + (1 - tau)*a
            falpha1 = F_U(x + alpha1*search)
        if abs(F_U(x + alpha1*search) - F_U(x + alpha2*search)) < epsilon:
            break
    return alpha1, falpha1






In [219]:
#Descenso más pronunciado
def StepestDescent(x,ep1,dx):
    
    falpha_prev = F_U(x)
    print('Initial function value = {0:.3f} '.format(falpha_prev))
    print('Iter o.\t x-vector \tf(x)\t Deriv ')
    print('------------------------------------------')
    for i in range(0, 3000):
        grad = gradiente(x,dx)
        Si = -grad
        
        alpha, falpha = GoldenSection(x, Si)
        if np.abs(falpha - falpha_prev) < ep1 or la.norm(grad) < ep1:
            break
        falpha_prev = falpha
        x = x + alpha*Si
        print('{0}\t[{1:.3f},{2:.3f}]\t{3:.3f}\t{4:.3f}'.format(i,x[0], x[1],falpha,la.norm(grad)))
    print('------------------------------------------')



In [220]:
#Método de Newtón
def Newton_Multivariate(x,ep1,ep2,dx):
    falpha_prev = F_U(x)
    print('Initial function value = {0:.2f} '.format(falpha_prev))
    print('Iter o.\t x-vector \tf(x)\t\t Deriv ')
    print('---------------------------------------------------')
    for i in range(1, 500):
        f_prev=F_U(x)
        grad =gradiente(x,dx)
        hess = Hs(x,dx)
        si=np.matmul(-1*la.inv(hess),grad.transpose())
        x = x + si.transpose()
        x=np.ndarray.flatten(x)
        f = F_U(x)
        print('{0}\t[{1:.3f},{2:.3f}]\t{3:.6f}\t{4:.6f}'.format(i,x[0], x[1],f_prev,la.norm(grad)))
        if abs(f-f_prev)<ep1 or la.norm(grad)<ep1:
            break;
       
    print('---------------------------------------------------')


In [221]:
#Método de Newtón Modificado

def Newton_Modificado(x,delta,ep1):
    fx_prev=F_U(x)
    print('Initial function value = {0:.4f} '.format(fx_prev))
    print('Iter o.\t x-vector \tf(x)\t\t Deriv ')
    print('---------------------------------------------------')
    for i in range(30):
        direccion=gradiente(x,delta)
        H=Hs(x,delta)
        direccion=np.atleast_2d(direccion)
        si=np.matmul(-la.inv(H),direccion.transpose())
        si = si.transpose()
        si=np.ndarray.flatten(si)
        alpha,fx_curr = GoldenSection(x, si)
        print(alpha)
        if abs(fx_curr-fx_prev)<ep1 or la.norm(direccion)<ep1:
            break;
        x = x +  alpha*si
        fx_prev=fx_curr
        print('{0}\t[{1:.3f},{2:.3f}]\t{3:.4f}\t{4:.4f}'.format(i,x[0], x[1],fx_prev,la.norm(direccion)))
        #print("%d\t %f %f\t %f \t%f \n" %(j,x[0],x[1],fx_curr,la.norm(dire)))
        print('---------------------------------------------------')

In [222]:
#Método de Levenberrg-Marquardt
def LevenberrgMarquardt(x,delta,ep1):
    lam = float(1e3)
    fx_prev=F_U(x)
    print('Initial function value = {0:.4f} '.format(fx_prev))
    print('Iter o.\t x-vector \tf(x)\t\t Deriv ')
    print('---------------------------------------------------')
    for i in range(30):
        fx_prev=F_U(x)
        direccion=gradiente(x,delta)
        H=Hs(x,delta)
        direccion=np.atleast_2d(direccion)
        si=np.matmul(-la.inv(H+lam*np.eye(len(x))),direccion.transpose())
        
        x = x + si.transpose() 
        x=np.ndarray.flatten(x)
        fx_curr = F_U(x)
        if fx_curr < fx_prev:
            lam = lam/2
        else:
            lam = 2*lam

        if abs(fx_curr-fx_prev)<ep1 or la.norm(direccion)<ep1:
            break;

    
        print('{0}\t[{1:.3f},{2:.3f}]\t{3:.6f}\t{4:.6f}'.format(i,x[0], x[1],fx_prev,la.norm(direccion)))

    print('{0}\t[{1:.3f},{2:.3f}]\t{3:.6f}\t{4:.6f}'.format(i,x[0], x[1],fx_curr,la.norm(direccion)))
    print('---------------------------------------------------')

In [223]:
#Llamar al descenso más pronunciado
x = [-3,2]
eps = 1e-6
dx = 1e-3
StepestDescent(x,eps,dx)

Initial function value = 1452.262 
Iter o.	 x-vector 	f(x)	 Deriv 
------------------------------------------
0	[0.095,0.023]	-2.704	1006.074
1	[0.170,0.141]	-5.278	37.036
2	[0.318,0.048]	-7.369	23.451
3	[0.375,0.138]	-8.773	26.656
4	[0.448,0.092]	-9.375	14.021
5	[0.470,0.127]	-9.583	10.071
6	[0.491,0.114]	-9.639	4.403
7	[0.497,0.123]	-9.652	2.509
8	[0.501,0.120]	-9.655	1.050
9	[0.503,0.122]	-9.656	0.554
10	[0.504,0.122]	-9.656	0.236
11	[0.504,0.122]	-9.656	0.125
12	[0.504,0.122]	-9.656	0.047
13	[0.504,0.122]	-9.656	0.027
14	[0.504,0.122]	-9.656	0.016
15	[0.504,0.122]	-9.656	0.044
16	[0.504,0.122]	-9.656	0.026
17	[0.504,0.122]	-9.656	0.016
18	[0.504,0.122]	-9.656	0.042
19	[0.504,0.122]	-9.656	0.025
20	[0.504,0.122]	-9.656	0.015
21	[0.504,0.122]	-9.656	0.041
22	[0.504,0.122]	-9.656	0.025
23	[0.504,0.122]	-9.656	0.015
24	[0.504,0.122]	-9.656	0.040
25	[0.504,0.122]	-9.656	0.024
26	[0.504,0.122]	-9.656	0.014
27	[0.504,0.122]	-9.656	0.039
28	[0.504,0.122]	-9.656	0.023
29	[0.504,0.122]	-9.65

In [224]:
#Llamar al método de Newtón
n_of_var = 2
x = [-3, 2]
ep1 = 1e-7
ep2 = 1e-7
delx = 1e-3
Newton_Multivariate(x,ep1,ep2,delx)

Initial function value = 1452.26 
Iter o.	 x-vector 	f(x)		 Deriv 
---------------------------------------------------
1	[-0.754,0.524]	1452.261884	1006.073751
2	[-0.362,-0.010]	44.243734	116.281440
3	[0.094,0.125]	8.398370	50.597488
4	[11.775,0.324]	-3.920442	21.420343
5	[1.042,0.093]	22007.141750	4076.915933
6	[0.640,0.142]	14.532812	102.745178
7	[0.524,0.122]	-8.478611	18.199370
8	[0.505,0.122]	-9.635114	2.212883
9	[0.504,0.122]	-9.656214	0.058979
10	[0.504,0.122]	-9.656230	0.000048
---------------------------------------------------


In [ ]:
#Llamar al método de Newtón Modificado
delta=1e-3 
ep1=1e-3 
x = [-3,2]
Newton_Modificado(x,delta,ep1)


In [ ]:
#Llamar al método de Levenberrg-Marquardt
delta=1e-3 
ep1=1e-3
x = [-3,2]


LevenberrgMarquardt(x,delta,ep1)

Initial function value = 1452.2619 
Iter o.	 x-vector 	f(x)		 Deriv 
---------------------------------------------------
0	[-2.384,1.604]	1452.261884	1006.073751
1	[-1.680,1.139]	815.737637	733.709354
2	[-1.104,0.705]	325.925171	429.113162
3	[-0.740,0.327]	102.058669	201.554316
4	[-0.444,0.133]	28.672897	86.884181
5	[-0.164,0.105]	8.324226	34.005458
6	[0.546,0.091]	1.185754	20.542003
7	[0.508,0.122]	-9.390497	11.361149
8	[0.505,0.122]	-9.656229	0.409016
---------------------------------------------------
